# Deep Neural Network for Fraud Detection: Application

## Packages

In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils_v3 import *
import csv
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict


%load_ext autoreload 
%autoreload 2

np.random.seed(1)

ModuleNotFoundError: No module named 'dnn_app_utils_v3'

## Dataset

The data for credit card fraud detection is from: 
https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
d = pd.read_csv("./creditcardfraud/creditcard.csv").values 
 
train = d[0:284800,:] 
test = d[284800:,:] 
train_x_orig = train[:,0:30] 
train_y = train[:,-1] 
test_x_orig = test[:,0:30] 
test_y = test[:,-1] 